In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pprint import pprint
import folium
from folium import plugins
from folium.plugins import HeatMap
%matplotlib inline
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import pygeohash as pgh
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plot_options = dict(figsize=(12,6))

In [2]:
conn = sqlite3.connect("Resources/FPA_FOD_20170508.sqlite")

In [3]:
df = pd.read_sql_query("SELECT * FROM Fires LIMIT 5;", conn)
df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [4]:
df = pd.read_sql_query("SELECT OBJECTID, LATITUDE, LONGITUDE, STAT_CAUSE_CODE, FIRE_SIZE, FIRE_YEAR FROM Fires;", conn)
df

,OBJECTID,LATITUDE,LONGITUDE,STAT_CAUSE_CODE,FIRE_SIZE,FIRE_YEAR
0,1,40.036944,-121.005833,9.0,0.10,2005
1,2,38.933056,-120.404444,1.0,0.25,2004
2,3,38.984167,-120.735556,5.0,0.10,2004
3,4,38.559167,-119.913333,1.0,0.10,2004
4,5,38.559167,-119.933056,1.0,0.10,2004
...,...,...,...,...,...,...
1880460,1880461,40.481637,-122.389375,13.0,0.01,2015
1880461,1880462,37.617619,-120.938570,9.0,0.20,2015
1880462,1880463,37.617619,-120.938570,13.0,0.10,2015
1880463,1880464,37.672235,-120.898356,13.0,2.00,2015


In [5]:
heat_df = df[["LATITUDE","LONGITUDE", "FIRE_YEAR", "FIRE_SIZE"]]
#heat_df = heat_df[heat_df["FIRE_YEAR"] == 1992]
heat_df = heat_df[heat_df["FIRE_SIZE"] > 1800]
heat_df = heat_df.dropna(axis=0, subset=["LATITUDE","LONGITUDE"])

# List comprehension to make out list of lists
heat_data = [[row["LATITUDE"],row["LONGITUDE"]] for index, row in heat_df.iterrows()]
print(heat_df.shape[0], " elements")
del heat_df

7666  elements


In [6]:

map_ = folium.Map(location=[df["LATITUDE"].mean(), df["LONGITUDE"].mean()],
                    tiles = "Stamen Terrain",                    
                    zoom_start = 3) 

# Plot it on the map
HeatMap(heat_data, min_opacity=.4, max_val=.8).add_to(map_)

# Display the map
map_

<ipython-input-6-93036e2a8958>:6: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, min_opacity=.4, max_val=.8).add_to(map_)


In [7]:

heat_time_df = df[["LATITUDE","LONGITUDE", "FIRE_YEAR", "FIRE_SIZE"]]
heat_time_df = heat_time_df[heat_time_df["FIRE_SIZE"] > 1500]
heat_time_df = heat_time_df.dropna(axis=0, subset=["LATITUDE","LONGITUDE"])

# List comprehension to make out list of lists
heat_time_data = [
    [
        [row['LATITUDE'],row['LONGITUDE']] for index, row in heat_time_df[heat_time_df["FIRE_YEAR"] == year].iterrows()
    ] for year in range(1992,2016)
]
print(heat_time_df.shape[0], " elements")
del heat_time_df

8504  elements


In [8]:
map_ = folium.Map(location=[df["LATITUDE"].mean()+3.3, df["LONGITUDE"].mean()-8.5],                  
                    tiles = "Stamen Terrain",
                    zoom_start = 4.5) 
# Plot it on the map
hm = plugins.HeatMapWithTime(
    heat_time_data,
    auto_play=True,
    max_opacity=0.6,
    name="HeatMap",
    index=list(range(1992,2016))
)
hm.add_to(map_)

map_